# インポート

In [2]:
import create_population
import numpy as np
import pandas as pd
import preprocessing
from pathlib import Path
import create_population
from feature_producing import FeatureCreator
from training import Trainer
from feature_producing_prediction import PredictionFeatureCreator
import predictioning
%load_ext autoreload


In [9]:
%autoreload

# 学習母集団の作成
期間を指定すると、学習母集団である（日付、race_id、horse_id）の組みを作る

In [4]:
population = create_population.create(
    from_ ="2015-01-01",
    to_ = "2024-12-31",
    race_info_filename="race_info15-24.csv",
    results_filename="results15-24.csv",
)

In [5]:
# 各年の数を確認
population["date"].dt.year.value_counts().sort_index()

date
2015    49992
2016    50076
2017    49299
2018    48604
2019    47574
2020    48282
2021    47821
2022    47220
2023    47672
2024    46549
Name: count, dtype: int64

# データ加工

In [2]:
import preprocessing
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
%autoreload

In [14]:
#レース結果テーブルの前処理
results_preprocessd = preprocessing.process_results(
    input_filename="results15-24.csv",
    output_filename="results15-24.csv",
)

In [15]:
# 重複チェック
results_preprocessd.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [16]:
# 欠損チェック
results_preprocessd.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
rank           0
umaban         0
wakuban        0
tansho_odds    0
popularity     0
impost         0
sex            0
age            0
weight         0
weight_diff    0
n_horses       0
time           0
time_rank      0
dtype: int64

In [17]:
# 馬の過去成績テーブルの前処理
horse_results_preprocessd = preprocessing.process_horse_results(
    input_filename="horse_results15-24.csv",
    output_filename="horse_results15-24.csv",
)

/home/load0/競馬AI/keiba_race_ai/keiba_ai/3.1.0/src/preprocessing.py:120: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_dir / input_filename, sep="\t").query(


In [18]:
horse_results_preprocessd

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses,time,win,rentai,show,place
0,2021101429,2024-11-09,9.0,0.0,0.2,0.0,1,1400,0.0,4.0,14.0,81.8,0,0,0,5.0
1,2021101429,2024-10-06,1.0,1140.0,0.0,1.0,1,1400,0.0,3.0,13.0,79.7,1,1,1,5.0
2,2021101429,2024-09-01,2.0,625.2,0.4,0.0,1,1400,0.0,5.0,7.0,82.0,0,1,1,4.0
3,2021101429,2024-08-10,2.0,629.2,0.0,0.0,1,1400,0.0,5.0,14.0,80.3,0,1,1,4.0
4,2021101429,2024-04-06,13.0,0.0,0.9,1.0,1,1600,1.0,7.0,16.0,95.3,0,0,0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198878,2015100692,2018-04-14,1.0,21.0,0.0,2.0,0,1300,1.0,NaN,9.0,84.4,1,1,1,NaN
1198879,2015100692,2018-03-31,1.0,20.0,0.0,0.0,0,900,0.0,NaN,10.0,55.2,1,1,1,NaN
1198880,2015100692,2018-03-11,4.0,1.0,0.5,0.0,0,1300,2.0,NaN,9.0,84.8,0,0,0,NaN
1198881,2015100692,2018-01-30,9.0,0.0,3.9,0.0,0,1500,1.0,NaN,11.0,100.5,0,0,0,NaN


In [19]:
# 形式、欠損値を確認
horse_results_preprocessd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1180504 entries, 0 to 1198882
Data columns (total 16 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   horse_id      1180504 non-null  int64         
 1   date          1180504 non-null  datetime64[ns]
 2   rank          1180504 non-null  float64       
 3   prize         1180504 non-null  float64       
 4   rank_diff     1179312 non-null  float64       
 5   weather       1179500 non-null  float64       
 6   race_type     1180504 non-null  int64         
 7   course_len    1180504 non-null  int64         
 8   ground_state  1180499 non-null  float64       
 9   race_class    624946 non-null   float64       
 10  n_horses      1180359 non-null  float64       
 11  time          1179971 non-null  float64       
 12  win           1180504 non-null  int64         
 13  rentai        1180504 non-null  int64         
 14  show          1180504 non-null  int64         
 15  pla

In [20]:
# 欠損チェック
horse_results_preprocessd.isnull().sum()

horse_id             0
date                 0
rank                 0
prize                0
rank_diff         1192
weather           1004
race_type            0
course_len           0
ground_state         5
race_class      555558
n_horses           145
time               533
win                  0
rentai               0
show                 0
place           656529
dtype: int64

In [16]:
%autoreload

In [21]:
# レース情報テーブルの加工
race_info_preprocess  = preprocessing.process_race_info(
    input_filename = "race_info15-24.csv",
    output_filename= "race_info15-24.csv",
)

In [22]:
race_info_preprocess

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place,month,sin_date,cos_date
0,201801010101,2018-07-28,1,0.0,1800,0,0,1,1,7,0.561693,0.101175
1,201801010102,2018-07-28,0,0.0,1000,0,0,1,1,7,0.561693,0.101175
2,201801010103,2018-07-28,0,0.0,2400,0,0,1,1,7,0.561693,0.101175
3,201801010104,2018-07-28,0,0.0,1700,0,0,1,1,7,0.561693,0.101175
4,201801010105,2018-07-28,1,0.0,1500,0,0,0,1,7,0.561693,0.101175
...,...,...,...,...,...,...,...,...,...,...,...,...
34542,201709050908,2017-12-28,1,0.0,1600,1,0,2,9,12,0.944121,1.998438
34543,201709050909,2017-12-28,0,0.0,1400,1,1,3,9,12,0.944121,1.998438
34544,201709050910,2017-12-28,1,0.0,1600,1,0,3,9,12,0.944121,1.998438
34545,201709050911,2017-12-28,0,0.0,1800,1,1,5,9,12,0.944121,1.998438


In [23]:
# 欠損チェック
race_info_preprocess.isnull().sum()

race_id            0
date               0
race_type          0
around          1253
course_len       263
weather            0
ground_state       0
race_class         0
place              0
month              0
sin_date           0
cos_date           0
dtype: int64

# 特徴量作成

In [ ]:
from feature_producing import FeatureCreator
%load_ext autoreload

In [ ]:
%autoreload

In [24]:
fp = FeatureCreator()
features = fp.create_features()

agg_horse_n_races_relative:   0%|          | 0/5 [00:00<?, ?it/s]

running agg_interval()...
running agg_jockey()...
running agg_trainer()...


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_sire()...
running cross_feature()...
merging all features...


In [89]:
# 重複チェック
features.duplicated(subset=["race_id", "horse_id"]).sum()

0

# 学習

In [ ]:
from training import Trainer
%load_ext autoreload

In [97]:
%autoreload

In [ ]:
trainer = Trainer()
%load_ext autoreload

In [ ]:
trainer = Trainer(config_filepath="config.yaml")
evaluation_df = trainer.run(
    valid_start_date="2022-10-01",
    test_start_date="2023-01-01",
    importance_filename="importance_new.csv",
    evaluation_filename="evaluation_new.csv"

[100]	training's binary_logloss: 0.153178	valid_1's binary_logloss: 0.206081


# 精度評価

In [1]:
from evaluationing import Evaluator
%load_ext autoreload

In [8]:
%autoreload

In [2]:
evaluator = Evaluator()

In [ ]:
evaluator = Evaluator(evaluation_filename="evaluation_5year.csv")


# 予測

## 事前準備
**当日出走馬が確定した時点**で実行できる

In [6]:
# 当日出走馬の過去成績テーブルの前処理
horse_results_preprocessed = preprocessing.process_horse_results(
    input_filename="horse_results_prediction.csv",
    output_filename="horse_results_prediction.csv"
)

In [ ]:
# 当日出走馬の血統テーブルの前処理
peds_preprocessed = preprocessing.process_peds(
    input_filename="peds_prediction.csv",
    output_filename="peds_prediction.csv"
)

In [7]:
pfc = PredictionFeatureCreator(population_filename="population_5year.csv")
# 過去成績集計は事前に行うことができる
pfc.create_baselog()
pfc.agg_horse_n_races()
pfc.agg_horse_n_races_relative()
# pfc.agg_interval()

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\common\\data\\prediction_population\\population_5year.csv'

## 当日の予測処理
レース直前出走直前に実行する

In [ ]:
# 特徴量の更新
features = pfc.create_features(
    race_id="202405030101",  # 予測するレースidを指定
    skip_agg_horse=True  # 事前に集計した場合はスキップできる
)

fetching shubuta page html...


/Users/chiyoda/repositories/keibaAI-v3-yt/v3_1_0/src/feature_engineering_prediction.py:183: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


agg_horse_per_course_len:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_ground_state_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_class:   0%|          | 0/6 [00:00<?, ?it/s]

agg_horse_per_race_type:   0%|          | 0/6 [00:00<?, ?it/s]

running agg_jockey()...
running agg_trainer()...
running agg_sire()...
running cross_feature()...
merging all features...


In [ ]:
# 予測
prediction.predict(features)

,race_id,umaban,tansho_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
2,202405030101,3,121.9,8,0.005624,0.008065,0.685545,0.983065
0,202405030101,1,3.1,1,0.309638,0.306667,0.959879,0.950667
8,202405030101,9,7.5,5,0.112428,0.125828,0.843211,0.943709
3,202405030101,4,3.5,2,0.234721,0.244898,0.821522,0.857143
6,202405030101,7,6.4,4,0.125690,0.129032,0.804418,0.825806
9,202405030101,10,19.9,7,0.047463,0.037303,0.944514,0.742324
1,202405030101,2,5.1,3,0.160924,0.144981,0.820712,0.739405
4,202405030101,5,10.0,6,0.086425,0.068456,0.864247,0.684564
7,202405030101,8,163.3,10,0.004097,0.002841,0.668959,0.463920
5,202405030101,6,146.9,9,0.005314,0.002841,0.780582,0.417330


In [ ]:
# オッズ除外モデルでの予測
prediction.predict(
    features,
    model_filename="model_odds_removed.pkl",
    calibration_model_filename="calibration_model_odds_removed.pkl",
    config_filepath="config_odds_removed.yaml"
)

,race_id,umaban,tansho_odds,popularity,pred,pred_calibrated,expect_return,expect_return_calibrated
5,202405030101,6,146.9,9,0.052190,0.038741,7.666716,5.691041
7,202405030101,8,163.3,10,0.041249,0.033053,6.735890,5.397591
2,202405030101,3,121.9,8,0.051555,0.035714,6.284495,4.353571
8,202405030101,9,7.5,5,0.121707,0.113924,0.912803,0.854430
4,202405030101,5,10.0,6,0.073797,0.081365,0.737972,0.813648
9,202405030101,10,19.9,7,0.041788,0.033053,0.831578,0.657759
1,202405030101,2,5.1,3,0.063697,0.066667,0.324854,0.340000
3,202405030101,4,3.5,2,0.089735,0.081365,0.314071,0.284777
0,202405030101,1,3.1,1,0.072417,0.081365,0.224494,0.252231
6,202405030101,7,6.4,4,0.048026,0.033053,0.307369,0.211541
